In [39]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import ftplib
from io import BytesIO
from PIL import Image, ImageDraw
import os
import ftplib
import pandas as pd
from glob import glob
import imageio.v3 as iio


In [40]:
#defining start and end time
start = '2023-08-19'
end = '2023-08-24'
var = 'tcwv'

#making a directory
OutputDirName = str(start + '_' + end + var)

os.mkdir(OutputDirName)

In [41]:
#loading data
d = xr.open_dataset('/Users/landergreenway/Downloads/School/JuniorYr/EV333_AtmosphericDynamics/python/FinalProject/atmorivers/ERA5_SouthAmerica_2023_hourly_tcwv.nc')

#subsetting data to average over i
d = d.sel(time = slice(start, end))
d = d['tcwv']

#defining number of frames in given time period
num = len(d['time'])
print(num)
print(d)

144
<xarray.DataArray 'tcwv' (time: 144, latitude: 181, longitude: 341)>
[8887824 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 -140.0 -139.8 -139.5 ... -55.5 -55.25 -55.0
  * latitude   (latitude) float32 -15.0 -15.25 -15.5 ... -59.5 -59.75 -60.0
  * time       (time) datetime64[ns] 2023-08-19 ... 2023-08-24T23:00:00
Attributes:
    units:          kg m**-2
    long_name:      Total column vertically-integrated water vapour
    standard_name:  lwe_thickness_of_atmosphere_mass_content_of_water_vapor


In [48]:
#data vis and projection
cmap = cmocean.cm.diff_r
lev = np.arange(0, 30, 0.25)
proj = ccrs.PlateCarree(central_longitude=-77)

#creating loop
for i in range(0, num):

    fig = plt.figure(figsize = (9, 4.5), dpi = 300)
    ax = plt.axes(projection = proj)
    
    # filled contour map of TPWV
    d[i].plot.contourf(
        x = 'longitude',
        y = 'latitude',
        ax=ax,
        transform=ccrs.PlateCarree(),
        levels=lev,
        extend='both',
        colors=cmap,
        add_colorbar=True,
        cbar_kwargs = {"label":"Total Precipitable Water Vapor (kg/m^2)"})
    # add coastlines
    ax.coastlines(
       resolution='50m')  #Currently can be one of “110m”, “50m”, and “10m”.

# add coastlines masking the land in lightgray
  #  ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '110m', edgecolor='k', facecolor='lightgray'))

# add grid lines
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True,
                  linewidth=1,
                  color='gray',
                  alpha=0.5,
                  linestyle='--')
    gl.right_labels = None
    gl.top_labels = None 

#extract year, month, day, and hour from the time series using splits, time format YYY-MM-DD-T00:00:00.000000000
    times = d['time'][i].values.astype('datetime64[s]')
    yr = str(np.datetime64(times, 'Y').astype(str))
    mon = str(np.datetime64(times, 'M').astype(str)).split('-')[1]
    day = str(np.datetime64(times, 'D').astype(str)).split('-')[2]
    hr = str(np.datetime64(times, 'h').astype(str)).split('T')[1].split(':')[0]

# add title
    ax.set_title("Total Column Water Vapor on "+ day + '/' + mon + ', ' + yr + ' at ' + hr + ':00')
#constructing filename
    name = OutputDirName + f'/RiversTest{var}_{yr}-{mon}-{day}-{hr}.png'

    #saveFolder = '/Users/landergreenway/Downloads/School/JuniorYr/EV333_AtmosphericDynamics/python/FinalProject/atmorivers/AtmoRiversFrames'
    
    
# save figure 
    fig.savefig(name, facecolor = 'white', transparent = False, bbox_inches ='tight')
   # fig.savefig(name, facecolor = 'white', transparent = False, bbox_inches ='tight')

    plt.close(fig)
    #print(name)
     
    

In [46]:
#Generating GIF from .png files
files = sorted(glob(OutputDirName + '/RiversTest*.png'))
files


['2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-00.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-01.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-02.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-03.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-04.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-05.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-06.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-07.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-08.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-09.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-10.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-11.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-12.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-13.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-14.png',
 '2023-08-19_2023-08-24tcwv/RiversTesttcwv_2023-08-19-1

In [49]:
#Generating GIF from .png files
#files = sorted(glob('OutputDirName/RiversTest*.png'))
#files

#creating array
images = [ ] 

#setting loop to append files to images, create gif file
for filename in files:
        images.append(iio.imread(filename))

iio.imwrite('AtmoRivers_'+ var + start + 'to' + end + '.gif', images, duration = 100, loop = 0)


In [37]:
#Making GIFs

#directory = '/AtmoriversIVTFrames/'
#filenames = 'RiversTestIVT*.png'

'''
files = sorted(glob('OutputDirName/RiversTestTCWV*.png'))
files

'''

"\nfiles = sorted(glob('OutputDirName/RiversTestTCWV*.png'))\nfiles\n\n"

In [38]:
'''
dirName = str(start + '_' + end)

os.mkdir(dirName)
'''

"\ndirName = str(start + '_' + end)\n\nos.mkdir(dirName)\n"